In [1]:
from scratchpad_radcad import model, simulation, experiment
from radcad import Experiment, Engine
import pandas as pd
import openai
# This model is taken from Danilo's (@danlessa) radcad example model. 
# All thanks to CadLabs and the contributors to radcad and cadcad.
# https://github.com/CADLabs/radCAD/tree/master/examples.

In [2]:
# read the openaikey from .env file
import os
from dotenv import load_dotenv
load_dotenv()
openai_key = os.getenv("OPENAI_KEY")


In [3]:
from cadcad_gpt import CadCAD_GPT

In [4]:
cadcad_gpt = CadCAD_GPT(openai_key, model, simulation, experiment)

In [5]:
def A_B_test(par1, val1, par2, val2, natural_language_request):
    """
    Perform an A/B test by updating the parameters par1 and par2 with values val1 and val2 respectively.
    Runs the experiment and returns the result of the A/B test.

    Parameters:
    par1 (str): The name of the first parameter to update.
    val1 (float): The value to set for the first parameter.
    par2 (str): The name of the second parameter to update.
    val2 (float): The value to set for the second parameter.
    natural_language_request (str): The natural language request that tests a metric on the dataframe generated by the A/B test. eg: which subset has the max value of prey_population?

    Returns:
    str: The code to run for further analysis.
    """
    initial_par1_value = simulation.model.params[par1][0]
    initial_par2_value = simulation.model.params[par2][0]
    
    simulation.model.params.update({
        par1: [val1, initial_par1_value],
        par2: [initial_par2_value, val2]
        })
    
    experiment = Experiment(simulation)
    experiment.engine = Engine()
    result = experiment.run()
    # Convert the results to a pandas DataFrame
    df = pd.DataFrame(result)

    systemprompt = f'''You are a python pandas helper. You are working with a dataframe called df and it has the following columns: {model.state.keys()}. The subset column 0 and 1 are to be compared on the metric that is given by the user. The code must print a string at the end explaining the comparision. Only output the python output enclosed in ``` backticks.'''
    completion = openai.ChatCompletion.create(
            model="gpt-4-1106-preview",
            max_tokens=500,
            messages=[
                {
                "role": "system",
                "content": f"{systemprompt}"
                },
                {
                "role": "user",
                "content": f"{natural_language_request}"
                }
            ],
            temperature=0,
            top_p=0.5,
        )
    answer = completion.choices[0].message.content
    code_to_run = answer.split('```')[1].replace('python', '')
    print(code_to_run)
    if 'df' in code_to_run:
        exec(code_to_run)

    return None

In [6]:
print(A_B_test('prey_reproduction_rate', 0.1, 'predator_interaction_factor', 0.2, 'which subset has the max value of prey_population?'))


# Group by 'subset' and calculate the max 'prey_population' for each subset
max_prey_population = df.groupby('subset')['prey_population'].max()

# Find the subset with the maximum 'prey_population'
max_subset = max_prey_population.idxmax()
max_value = max_prey_population.max()

# Print the result
print(f"Subset {max_subset} has the maximum value of prey_population, which is {max_value}.")

Subset 1 has the maximum value of prey_population, which is 105.0.
None


In [7]:
def param_sweep(par1, range, natural_language_request):
    """
    Perform a parameter sweep by updating the parameter par1 with values in range.
    Runs the experiment and returns the result of the parameter sweep.

    Parameters:
    par1 (str): The name of the parameter to update.
    range (list): The list of values to set for the parameter.
    natural_language_request (str): The natural language request that tests a metric on the dataframe generated by the parameter sweep. eg: which subset has the max value of prey_population?

    Returns:
    str: The code to run for further analysis.
    """
    
    simulation.model.params.update({
        par1: range
        })
    
    experiment = Experiment(simulation)
    experiment.engine = Engine()
    result = experiment.run()
    # Convert the results to a pandas DataFrame
    df = pd.DataFrame(result)

    systemprompt = f'''You are a python pandas helper. 
    You are working with a dataframe called df and it has the following columns: {model.state.keys()}. 
    The subset column values represent different experiments like 0,1,2.. and they are to be compared on the metric that is given by the user.
    The code must print the intermediate variables and as well as a string at the end explaining the comparision. Only output the python output enclosed in ``` backticks.'''
    completion = openai.ChatCompletion.create(
            model="gpt-4-1106-preview",
            max_tokens=500,
            messages=[
                {
                "role": "system",
                "content": f"{systemprompt}"
                },
                {
                "role": "user",
                "content": f"{natural_language_request}"
                }
            ],
            temperature=0,
            top_p=0.5,
        )
    answer = completion.choices[0].message.content
    code_to_run = answer.split('```')[1].replace('python', '')
    print(code_to_run)
    if 'df' in code_to_run:
        exec(code_to_run)

    return None

In [8]:
param_sweep('prey_reproduction_rate', [0, 0.1, 0.2, 0.3, 0.4, 0.5, 5], 'In which subsets does the prey_population not go extinct?')


# Group by 'subset' and find the minimum 'prey_population' for each subset
min_prey_population_per_subset = df.groupby('subset')['prey_population'].min()

# Print the intermediate variable
print("Minimum prey population per subset:")
print(min_prey_population_per_subset)

# Find subsets where the prey population does not go extinct (min prey population > 0)
non_extinct_subsets = min_prey_population_per_subset[min_prey_population_per_subset > 0].index.tolist()

# Print the subsets where the prey population does not go extinct
print("Subsets where the prey population does not go extinct:")
print(non_extinct_subsets)

# Explanation string
explanation = f"The prey population does not go extinct in the following subsets: {non_extinct_subsets}"
print(explanation)

Minimum prey population per subset:
subset
0    45.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
Name: prey_population, dtype: float64
Subsets where the prey population does not go extinct:
[0]
The prey population 

In [13]:
# range = [i*0.1 for i in range(30)]

# simulation.model.params.update({
#     'prey_reproduction_rate': [0.1,0.2,0.3,0.4,0.5,0.6]
#     })

# experiment = Experiment(simulation)
# experiment.engine = Engine()
# result = experiment.run()
# # Convert the results to a pandas DataFrame
# df = pd.DataFrame(result)

In [20]:
param_sweep('prey_reproduction_rate', 'range', 'In which subsets does the prey_population have a mean value of less than 200?')


# Group by 'subset' and calculate the mean of 'prey_population'
subset_means = df.groupby('subset')['prey_population'].mean()

# Filter the groups with a mean less than 200
subsets_with_low_mean = subset_means[subset_means < 200].index.tolist()

# Print the result
print(f"Subsets with a mean prey_population of less than 200: {subsets_with_low_mean}")

Subsets with a mean prey_population of less than 200: [0, 1]


In [21]:
# range = [i*0.1 for i in range(30)]

simulation.model.params.update({
    'prey_reproduction_rate': [0.1,0.2,0.3,0.4,0.5,0.6]
    })

experiment = Experiment(simulation)
experiment.engine = Engine()
result = experiment.run()
# Convert the results to a pandas DataFrame
df = pd.DataFrame(result)

In [23]:
subset_means = df.groupby('subset')['prey_population'].mean()

# Filter the groups with a mean less than 200
subsets_with_low_mean = subset_means[subset_means < 200].index.tolist()

# Print the result
print(f"Subsets with a mean prey_population of less than 200: {subsets_with_low_mean}")


Subsets with a mean prey_population of less than 200: [0, 1]


In [24]:
subset_means

subset
0      80.190809
1      99.000999
2    3046.802198
3    4066.255744
4    4327.371628
5    4461.342657
Name: prey_population, dtype: float64

In [9]:
cadcad_gpt('can you change the prey reproduction rate to 0.1 and tell me the mean of the prey_population column?')

Planner Agent:
I have made a plan to follow:
Step 1 use change_param to change the prey reproduction rate to 0.1
Step 2 use analysis_agent to calculate the mean of the prey_population column


Executor Agent:
Thought: My task is to use change_param to change the prey reproduction rate to 0.1
Action: I should call change_param function with these {'param': 'prey_reproduction_rate', 'value': 0.1} arguments.
Observation: 
new prey_reproduction_rate value is 0.1 and the simulation dataframe is updated
Executor Agent:
Thought: My task is to use analysis_agent to calculate the mean of the prey_population column
Action: I should call analysis_agent function with these {'question': 'What is the mean of the prey_population column?'} arguments.


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['prey_population'].mean()"}`


80.19080919080919The mean of the `prey_population` column is approximately 80.19.

> Finished chain.
Observation: 
The mean of the `pre

In [11]:
cadcad_gpt('can you change the prey reproduction rate to 0.2 and tell me the mean of the prey_population column?')

Planner Agent:
I have made a plan to follow:
Step 1 use function change_param to change the prey reproduction rate to 0.2
Step 2 use function analysis_agent to get the mean of the prey_population column.


Executor Agent:
Thought: My task is to use function change_param to change the prey reproduction rate to 0.2
Action: I should call change_param function with these {'param': 'prey_reproduction_rate', 'value': 0.2} arguments.
Observation: 
new prey_reproduction_rate value is 0.2 and the simulation dataframe is updated
Executor Agent:
Thought: My task is to use function analysis_agent to get the mean of the prey_population column.
Action: I should call analysis_agent function with these {'question': 'What is the mean of the prey_population column?'} arguments.


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['prey_population'].mean()"}`


99.000999000999The mean of the `prey_population` column is approximately 99.001.

> Finished chain.
Observati